In [9]:
pip install pandas numpy scikit-learn nltk

In [10]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report



In [11]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\Vishnu
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import os

reqd_file_path = os.path.join(os.getcwd(),'datasets','MovieReviewTrainingDatabase.csv')

print(reqd_file_path)

d:\sandbox\python\experiments\machine-learning-experiments\Project-Machine-Learning\datasets\MovieReviewTrainingDatabase.csv


In [13]:

df = pd.read_csv(reqd_file_path)

print(df.head())

  sentiment                                             review
0  Positive  With all this stuff going down at the moment w...
1  Positive  'The Classic War of the Worlds' by Timothy Hin...
2  Negative  The film starts with a manager (Nicholas Bell)...
3  Negative  It must be assumed that those who praised this...
4  Positive  Superbly trashy and wondrously unpretentious 8...


In [14]:
df

,sentiment,review
0,Positive,With all this stuff going down at the moment w...
1,Positive,'The Classic War of the Worlds' by Timothy Hin...
2,Negative,The film starts with a manager (Nicholas Bell)...
3,Negative,It must be assumed that those who praised this...
4,Positive,Superbly trashy and wondrously unpretentious 8...
...,...,...
24995,Negative,It seems like more consideration has gone into...
24996,Negative,I don't believe they made this film. Completel...
24997,Negative,"Guy is a loser. Can't get girls, needs to buil..."
24998,Negative,This 30 minute documentary Buñuel made in the ...


Preprocess the Data

In [15]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text


In [16]:
print(df.head())

  sentiment                                             review
0  Positive  With all this stuff going down at the moment w...
1  Positive  'The Classic War of the Worlds' by Timothy Hin...
2  Negative  The film starts with a manager (Nicholas Bell)...
3  Negative  It must be assumed that those who praised this...
4  Positive  Superbly trashy and wondrously unpretentious 8...


Let's reduce the sample size? 25,000 rows is too much.

In [17]:
df_micro = df.sample(n=200, random_state=42)

In [20]:
df_micro

,sentiment,review
6868,Negative,I read that 'There's a Girl in My Soup' came o...
24016,Positive,This film pulls you in from the get-go because...
9668,Negative,From the awful death scenes to guns that fire ...
13640,Positive,I saw that movie few days ago. This movie is s...
14018,Negative,For a film with so much promise it was disappo...
...,...,...
1747,Positive,Kate Miller (Angie Dickinson) is having proble...
323,Positive,I have been a Hindi movie buff since the age o...
10296,Positive,D.W. Griffith could have made any film he want...
12750,Negative,Saw this film ran in the wee hours on TCM. Sev...


In [21]:
df_micro['cleaned_text'] = df_micro['review'].apply(preprocess_text)

In [31]:
print(df_micro)

      sentiment                                             review  \
6868   Negative  I read that 'There's a Girl in My Soup' came o...   
24016  Positive  This film pulls you in from the get-go because...   
9668   Negative  From the awful death scenes to guns that fire ...   
13640  Positive  I saw that movie few days ago. This movie is s...   
14018  Negative  For a film with so much promise it was disappo...   
...         ...                                                ...   
1747   Positive  Kate Miller (Angie Dickinson) is having proble...   
323    Positive  I have been a Hindi movie buff since the age o...   
10296  Positive  D.W. Griffith could have made any film he want...   
12750  Negative  Saw this film ran in the wee hours on TCM. Sev...   
10012  Positive  This is another gem of a stand up show from Ed...   

                                            cleaned_text  
6868   read girl soup came peter sellers low period w...  
24016  film pulls get go grabs attention 

In [33]:
X = df_micro['cleaned_text']
print(X)

6868     read girl soup came peter sellers low period w...
24016    film pulls get go grabs attention acknowledgin...
9668     awful death scenes guns fire without making so...
13640    saw movie days ago movie great makes feel want...
14018    film much promise disappointing thinly plotted...
                               ...                        
1747     kate miller angie dickinson problems marriage ...
323      hindi movie buff since age 4 never life watche...
10296    w griffith could made film wanted enormous fin...
12750    saw film ran wee hours tcm several problems fi...
10012    another gem stand show eddie izzard cannot fai...
Name: cleaned_text, Length: 200, dtype: object


In [34]:
y = df_micro['sentiment']
print(y)

6868     Negative
24016    Positive
9668     Negative
13640    Positive
14018    Negative
           ...   
1747     Positive
323      Positive
10296    Positive
12750    Negative
10012    Positive
Name: sentiment, Length: 200, dtype: object


Split the Dataset

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [36]:
print(X_train.shape)

(160,)


In [37]:
print(y_train.shape)

(160,)


In [38]:
print(X_train.shape[0], y_train.shape[0])

160 160


 Build the Naive Bayes Model

In [39]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [40]:
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

Predict test data

In [41]:
y_pred = model.predict(X_test)

Evaluate the model

In [42]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')

Accuracy: 65.00%


In [43]:

print('\nClassification report: \n\n', classification_report(y_test, y_pred))


Classification report: 

               precision    recall  f1-score   support

    Negative       0.58      0.95      0.72        19
    Positive       0.89      0.38      0.53        21

    accuracy                           0.65        40
   macro avg       0.73      0.66      0.63        40
weighted avg       0.74      0.65      0.62        40



Test with custom input 

In [44]:
review = ['This movie was absolutely fantastic! The storyline was gripping.']
print('Prediction: ', model.predict(review))

Prediction:  ['Positive']


In [45]:
review2 = ['The movie was shit']
print('Prediction: ', model.predict(review2))

Prediction:  ['Negative']
